In [ ]:
using Revise

using Kate

### Check Environment

In [ ]:
Kate.check_environment()

### Set Parameters

In [ ]:
sa = "test_high_quality"

# sa = "S2012_cat"

# fq1 = "../input/S2012_cat/S2012_R1.fastq.gz"

# fq2 = "../input/S2012_cat/S2012_R2.fastq.gz"

# is dna targeted
ta = false

# Number of jobs
n_jo = 8

# G of memory
meto = 40

# G of memory per job
mejo = 2;

In [ ]:
papr = dirname(@__DIR__)

paou = joinpath(papr, "output")

paip = joinpath(papr, "input")

pasa = joinpath(paip, sa)

pare = joinpath(paip, "reference")

pasn = "/opt/snpeff/snpEff/snpEff.jar";

### Check reads

In [ ]:
re_ = Kate.find_read(pasa)

In [ ]:
Kate.check_sequence(
    re_,
    joinpath(paou, sa, string("check_sequence_raw")),
    n_jo,
)

### Concatenate reads by strand

In [ ]:
Kate.concatenate_reads(re_, sa, paip)

### Process reads

In [ ]:
fq1 = "../input/test_high_quality/germ.1k.R1.fastq.gz"

fq2 = "../input/test_high_quality/germ.1k.R2.fastq.gz"

In [ ]:
process_dna_arguments = (
    joinpath(pare, "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz"),
    joinpath(pare, "chromosome.bed.gz"),
    joinpath(pare, "chrn_n.tsv"),
    n_jo,
    meto,
    mejo,
    pasn,
)

In [ ]:
Kate.process_germ_dna(
    fq1,
    fq2,
    ta,
    joinpath(paou, string(sa)),
    process_dna_arguments...,
)

# Benchmark

TODO: Refactor

In [ ]:
papa = dirname(@__DIR__)

using JSON: parse

project_json = parse(read(joinpath(papa, "project.json"), String))

paou = joinpath(papa, "output")

paip = joinpath(papa, "input")

reference_dir = joinpath(paip, "reference")

baseline_vcf = joinpath(paip, "high_confidence_HG002_4.1_GRCh38/HG002_GRCh38_1_22_v4.1_draft_benchmark.vcf.gz")

calls_vcf = joinpath(paou, "process_germ_dna_HG002_cat/find_variant/strelka/results/variants/variants.vcf.gz")

rtg_sdf_reference_dir = joinpath(paip, "GRCh38.sdf")

benchmark_paou = joinpath(paou, "process_germ_dna_HG002_cat/benchmark")

rtg_executable = "/opt/rtg/rtg-tools-3.11/rtg"

### Configure vcfeval

In [ ]:
run(`$rtg_executable`)

### Download Real Time Genomics prepared GRCH38 files
vcfeval requires reference files in RTG formats. RTG makes versions of these files for the popular reference genomes such as GRCh38, which will be downloaded below.

In [ ]:
if ispath(rtg_sdf_reference_dir)
    
    println("Skipping download because RTG GRCh38 data directory already exists:\n $rtg_sdf_reference_dir")

else

    run(pipeline(
            `wget -q -P ../input/ https://s3.amazonaws.com/rtg-datasets/references/GRCh38.sdf.zip`,
            `unzip -d ../input/ ../input/GRCh38.sdf.zip`,
            `rm ../input/GRCh38.sdf.zip`,
            ))
end

### Run vcfeval
This command comapares the VCF you generated (calls) to the highly confident VCF generated by Genome in a Bottle (baseline).

In [ ]:
run(`$rtg_executable vcfeval
    --baseline=$baseline_vcf
    --calls=$calls_vcf
    --template=$rtg_sdf_reference_dir
    --output=$benchmark_paou
    --threads=1
    `)

### See results
Read more about how to interpret vcfeval results [here](https://cdn.rawgit.com/RealTimeGenomics/rtg-tools/master/installer/resources/tools/RTGOperationsManual/rtg_command_reference.html#vcfeval).

In [ ]:
vcfeval_summary_file = open(joinpath(benchmark_paouput_dir, "summary.txt"))

vcfeval_summary = read(vcfeval_summary_file)

vcfeval_summary